In [2]:
import pandas as pd
data = pd.read_csv(r"C:\Users\FPTSHOP\Downloads\output (5).csv")

In [3]:
# Assuming 'data' is your DataFrame with coordinates

# Create a dictionary to map coordinates to indices
coord_to_index = {coord: i for i, coord in enumerate(data['node_start'].unique())}

# Create a reverse mapping to translate indices back to coordinates
index_to_coord = {i: coord for coord, i in coord_to_index.items()}


In [4]:
import ast

def invert_coordinates(coord):
    # Safely evaluate the string as a tuple
    coord_tuple = ast.literal_eval(coord)
    # Invert the coordinates
    return (coord_tuple[1], coord_tuple[0])

# Apply the inversion to your DataFrame
data['node_start'] = data['node_start'].apply(invert_coordinates)
data['node_end'] = data['node_end'].apply(invert_coordinates)
# Recreate coordinate-index mappings
coord_to_index = {coord: i for i, coord in enumerate(pd.unique(data[['node_start', 'node_end']].values.ravel('K')))}
index_to_coord = {i: coord for coord, i in coord_to_index.items()}

# Also, recompute any matrices or structures using the corrected coordinates


In [5]:
from scipy.sparse import lil_matrix

# Initialize sparse matrices
num_nodes = len(coord_to_index)
sparse_distance_matrix = lil_matrix((num_nodes, num_nodes))

# Populate the distance matrix
for _, row in data.iterrows():
    start_index = coord_to_index[row['node_start']]
    end_index = coord_to_index[row['node_end']]
    sparse_distance_matrix[start_index, end_index] = row['distance']


In [6]:
from scipy.sparse import save_npz, csr_matrix

# Convert your lil_matrix to csr_matrix for efficient storage
csr_distance_matrix = csr_matrix(sparse_distance_matrix)

# Save the sparse matrix
save_npz('distance_matrix.npz', csr_distance_matrix)

# If you have a heuristic matrix
# csr_heuristic_matrix = csr_matrix(heuristic)
# save_npz('heuristic_matrix.npz', csr_heuristic_matrix)


In [20]:
import numpy as np
from scipy.sparse import load_npz
import pickle
from queue import PriorityQueue
import math

csr_distance_matrix = load_npz('distance_matrix.npz')
# csr_heuristic_matrix = load_npz('heuristic_matrix.npz')  # Uncomment if you have a heuristic matrix


#Calculate haversine distance
def haversine(coord1, coord2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest_matrix_index(input_coord, index_to_coord_map):
    """Find the nearest point's index in the matrix to the input_coord."""
    min_distance = float('inf')
    nearest_index = None

    for index, coord in index_to_coord_map.items():
        distance = haversine(input_coord, coord)
        if distance < min_distance:
            min_distance = distance
            nearest_index = index

    return nearest_index



In [8]:
def UCS(start_coord, goal_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}  # Dictionary to count how many times each node is seen
    explored_routes = []  # List to store explored routes

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1

        if current == goal:
            break

        for next_node in range(csr_distance_matrix.shape[1]):
            if csr_distance_matrix[current, next_node] != 0:
                new_cost = cost_so_far[current] + csr_distance_matrix[current, next_node]
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost 
                    open_set.put((priority, next_node))
                    came_from[next_node] = current
                    explored_routes.append((current, next_node))  # Add route to the list

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            print(f"Path broken at {current}. No entry in 'came_from'.")
            break  # or handle the situation as required
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far[goal], node_seen_count, explored_routes

In [9]:
def greedy(start_coord, goal_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}  # Dictionary to count how many times each node is seen
    explored_routes = []  # List to store explored routes

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1

        if current == goal:
            break

        for next_node in range(csr_distance_matrix.shape[1]):
            if csr_distance_matrix[current, next_node] != 0:
                new_cost = cost_so_far[current] + csr_distance_matrix[current, next_node]
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = haversine(index_to_coord[next_node], goal_coord)
                    open_set.put((priority, next_node))
                    came_from[next_node] = current
                    explored_routes.append((current, next_node))  # Add route to the list

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            print(f"Path broken at {current}. No entry in 'came_from'.")
            break  # or handle the situation as required
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far[goal], node_seen_count, explored_routes


In [10]:
def a_star(start_coord, goal_coord):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}  # Dictionary to count how many times each node is seen
    explored_routes = []  # List to store explored routes

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1

        if current == goal:
            break

        for next_node in range(csr_distance_matrix.shape[1]):
            if csr_distance_matrix[current, next_node] != 0:
                new_cost = cost_so_far[current] + csr_distance_matrix[current, next_node]
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost + haversine(index_to_coord[next_node], goal_coord)
                    open_set.put((priority, next_node))
                    came_from[next_node] = current
                    explored_routes.append((current, next_node))  # Add route to the list

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            print(f"Path broken at {current}. No entry in 'came_from'.")
            break  # or handle the situation as required
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far[goal], node_seen_count, explored_routes


In [11]:
def a_star_plus(start_coord, goal_coord):
    base = 0.6*haversine(start_coord, goal_coord)
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))

    came_from = {start: None}
    cost_so_far = {start: 0}

    node_seen_count = {}  # Dictionary to count how many times each node is seen
    explored_routes = []  # List to store explored routes

    while not open_set.empty():
        current = open_set.get()[1]
        node_seen_count[current] = node_seen_count.get(current, 0) + 1

        if current == goal:
            break

        for next_node in range(csr_distance_matrix.shape[1]):
            if csr_distance_matrix[current, next_node] != 0:
                new_cost = cost_so_far[current] + csr_distance_matrix[current, next_node]
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    heuristic = haversine(index_to_coord[next_node], goal_coord)
                    if heuristic>=base:
                        priority = new_cost + haversine(index_to_coord[next_node], goal_coord)
                    else :
                        priority = new_cost + haversine(index_to_coord[next_node], goal_coord)*0.55
                    open_set.put((priority, next_node))
                    came_from[next_node] = current
                    explored_routes.append((current, next_node))  # Add route to the list

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        if current not in came_from:
            print(f"Path broken at {current}. No entry in 'came_from'.")
            break  # or handle the situation as required
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()

    return path, cost_so_far[goal], node_seen_count, explored_routes


In [12]:
def prune_open_set(open_set, memory_limit):
    if open_set.qsize() > memory_limit:
        # Keep the nodes with the lowest estimated total cost
        kept_nodes = [open_set.get() for _ in range(memory_limit)]
        open_set.queue.clear()
        for item in kept_nodes:
            open_set.put(item)

def a_star_memory_bounded(start_coord, goal_coord, memory_limit):
    start = coord_to_index[start_coord]
    goal = coord_to_index[goal_coord]

    open_set = PriorityQueue()
    open_set.put((0, start))  # (priority, node)

    came_from = {start: None}
    cost_so_far = {start: 0}

    while not open_set.empty():
        prune_open_set(open_set, memory_limit)  # Prune the open set to meet memory constraints

        current_priority, current = open_set.get()

        if current == goal:
            break  # Goal reached

        for next_node in range(csr_distance_matrix.shape[1]):
            if csr_distance_matrix[current, next_node] != 0:
                new_cost = cost_so_far[current] + csr_distance_matrix[current, next_node]
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    total_cost = new_cost + haversine(index_to_coord[next_node], goal_coord)
                    open_set.put((total_cost, next_node))
                    came_from[next_node] = current

    # Reconstruct path
    path = reconstruct_path(came_from, start, goal)
    return path, cost_so_far[goal]

def reconstruct_path(came_from, start, goal):
    path = []
    current = goal
    while current != start:
        path.append(index_to_coord[current])
        current = came_from[current]
    path.append(index_to_coord[start])
    path.reverse()
    return path


In [24]:
# Example usage
memory_limit = 100
start_coord = (21.0216642, 105.8388283)
goal_coord =( 21.0081752,105.8456702)
path, total_cost, node_seen_count, explored_routes = a_star(start_coord, goal_coord)

print("Path:", path)
print("Total Cost:", total_cost)
print("Node Seen Count:", node_seen_count)
print("Explored Routes:", explored_routes)

KeyError: 343

In [ ]:
import folium

def plot_search_visualization(path, node_seen_count, explored_routes, index_to_coord):
    if not path:
        raise ValueError("Path is empty, cannot visualize.")

    # Initialize the map centered around the start of the path
    m = folium.Map(location=path[0], zoom_start=14)

    # Add markers for the start and end of the path
    folium.Marker(path[0], popup='Start', icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(path[-1], popup='Goal', icon=folium.Icon(color='red')).add_to(m)

    # Add the true path
    folium.PolyLine(path, color='blue', weight=2.5, opacity=1).add_to(m)

    # Add markers for nodes seen
    for node, count in node_seen_count.items():
        if node in index_to_coord:
            coord = index_to_coord[node]
            folium.CircleMarker(coord, radius=5, color='orange', fill=True, fill_color='orange', popup=f'Seen {count} times').add_to(m)

    # Add lines for explored routes
    for route in explored_routes:
        start_node, end_node = route
        if start_node in index_to_coord and end_node in index_to_coord:
            start_coord = index_to_coord[start_node]
            end_coord = index_to_coord[end_node]
            folium.PolyLine([start_coord, end_coord], color='grey', weight=1, opacity=0.5).add_to(m)

    return m

# Example usage
# Assuming you have index_to_coord mapping and the other required data from your A* output
map_visualization = plot_search_visualization(path, node_seen_count, explored_routes, index_to_coord)
map_visualization.save('search_visualization.html')
